In [1]:
import pandas as pd
import numpy as np

In [5]:
# definir arquivo Pickle de entrada
arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf05.csv'

In [6]:
df = pd.read_pickle(arquivo)
df.head()

,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,121,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,206,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,1,233,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
FRANCISCO,1,246,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0
JOAO,1,225,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
df2 = df.copy()
df2['TEMP'] = df2[['SEXO', 'QTDE']].apply(lambda x: [x[0]] * x[1], axis=1)
df3 = df2.explode('TEMP').drop(['TEMP'], axis=1) #.reset_index()

# usar os dados expandidos invés dos agregados
df0 = df # guardar o dataframe original
df = df3 # apontar para o novo dataframe

In [8]:
X = df.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados originais:', X.shape, y.shape)

Forma dos dados originais: (47033, 16) (47033,)


In [9]:
# dividir dados em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Shapes: (31512, 16) (15521, 16) (31512,) (15521,)


In [10]:
# treinar com todos os dados disponíveis

X_train, y_train = X, y

print("Shapes:", X_train.shape, y_train.shape)

Shapes: (47033, 16) (47033,)


https://automl.github.io/auto-sklearn/master/manual.html

In [11]:
# https://automl.github.io/auto-sklearn/master/installation.html

#!curl -k https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install
#!pip3 install auto-sklearn

In [12]:
import autosklearn.classification

In [13]:
X_train.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0


In [15]:
y_train[:5]

NOME
MARIA    0
MARIA    0
MARIA    0
MARIA    0
MARIA    0
Name: SEXO, dtype: uint8

In [17]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=20*60,
    per_run_time_limit=2*60,
    n_jobs=-1,
    memory_limit=2048,
    seed=42,
    include_preprocessors=["no_preprocessing"]
)
automl

AutoSklearnClassifier(include_preprocessors=['no_preprocessing'],
                      memory_limit=2048, n_jobs=-1, per_run_time_limit=120,
                      seed=42, time_left_for_this_task=1200)

In [18]:
%%time
automl.fit(X_train, y_train)

CPU times: user 12min 26s, sys: 28.9 s, total: 12min 55s
Wall time: 20min 2s


AutoSklearnClassifier(include_preprocessors=['no_preprocessing'],
                      memory_limit=2048, n_jobs=-1, per_run_time_limit=120,
                      seed=42, time_left_for_this_task=1200)

In [19]:
print(automl.show_models())

[(0.180000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'adaboost', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'median', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'minmax', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:adaboost:algorithm': 'SAMME', 'classifier:adaboost:learning_rate': 0.22517938538440133, 'classifier:adaboost:max_depth': 7, 'classifier:adaboost:n_estimators': 254, 'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction': 0.0004228019256792407},
dataset_properties={
  'task': 1,
  'sparse': False,
  'multilabel': False,
  'multiclass': False,
  'target_type': 'classification',
  'signed': False})),
(0.140000, SimpleClassif

In [20]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: ddb1601b-c015-11eb-a486-4357eb61deda
  Metric: accuracy
  Best validation score: 0.999227
  Number of target algorithm runs: 319
  Number of successful target algorithm runs: 250
  Number of crashed target algorithm runs: 41
  Number of target algorithms that exceeded the time limit: 28
  Number of target algorithms that exceeded the memory limit: 0



In [21]:
losses_and_configurations = [
    (run_value.cost, run_key.config_id)
    for run_key, run_value in automl.automl_.runhistory_.data.items()
]
losses_and_configurations.sort()
print("Lowest loss:", losses_and_configurations[0][0])
print("Best configuration:", automl.automl_.runhistory_.ids_config[losses_and_configurations[0][1]])

Lowest loss: 0.0007731460601765328
Best configuration: Configuration:
  balancing:strategy, Value: 'none'
  classifier:__choice__, Value: 'adaboost'
  classifier:adaboost:algorithm, Value: 'SAMME'
  classifier:adaboost:learning_rate, Value: 0.22863898818862605
  classifier:adaboost:max_depth, Value: 8
  classifier:adaboost:n_estimators, Value: 259
  data_preprocessing:categorical_transformer:categorical_encoding:__choice__, Value: 'no_encoding'
  data_preprocessing:categorical_transformer:category_coalescence:__choice__, Value: 'minority_coalescer'
  data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction, Value: 0.00044177091870148513
  data_preprocessing:numerical_transformer:imputation:strategy, Value: 'mean'
  data_preprocessing:numerical_transformer:rescaling:__choice__, Value: 'minmax'
  feature_preprocessor:__choice__, Value: 'no_preprocessing'



In [22]:
# treinar o melhor modelo com todos os dados
automl.fit(X, y)

ValueError: {filepath} already exist. Different seeds should be provided for different jobs.

In [ ]:
# carregar dados de teste: definir arquivo Pickle de entrada
#arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf-preparado.pkl.bz2'

In [ ]:
df2 = pd.read_pickle(arquivo)
df2.head()

In [ ]:
df2.shape

In [ ]:
X_test = df2.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y_test = df2['SEXO'] # apenas a coluna alvo

In [ ]:
y_pred = automl.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score

print("Accuracy score: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

In [ ]:
from sklearn.metrics import confusion_matrix

print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))